## Test the model here [SumX](https://huggingface.co/Priyanshuchaudhary2425/Sumx-test-model)

## **Note:-** SumX is trained on very small dataset i.e only with 2k dataset due to compute power.

## 1. Loading datasets

In [1]:
!pip install transformers -U

In [5]:
!pip install datasets

In [6]:
from datasets import load_dataset

raw_dataset = load_dataset("cnn_dailymail", "3.0.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

## 2. EDA

In [8]:
raw_dataset.set_format("pandas")
cnn_train = raw_dataset["train"][:]
cnn_test = raw_dataset["test"][:]
cnn_val = raw_dataset["validation"][:]

In [9]:
cnn_train.head()

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [10]:
cnn_train.drop("id", axis=1, inplace=True)
cnn_test.drop("id", axis=1, inplace=True)
cnn_val.drop("id", axis=1, inplace=True)

In [11]:
cnn_train.head()

,article,highlights
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa..."
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non..."
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical..."


In [12]:
cnn_train.describe()

,article,highlights
count,287113,287113
unique,284005,282197
top,"SHANGHAI, China (CNN) -- If it weren't for th...",This page includes the show Transcript and the...
freq,3,83


In [13]:
cnn_train.isna().sum()

article       0
highlights    0
dtype: int64

In [14]:
cnn_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   article     287113 non-null  object
 1   highlights  287113 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


In [15]:
cnn_train = cnn_train.drop_duplicates(subset=["highlights"])

# Reset the index
cnn_train.reset_index(drop=True, inplace=True)

In [16]:
cnn_train.describe()

,article,highlights
count,282197,282197
unique,282187,282197
top,"Everett, Washington (CNN) -- After more than ...",Harry Potter star Daniel Radcliffe gets £20M f...
freq,2,1


In [17]:
cnn_train["article"][282195]

"Most men enjoy a good pint of lager or real ale while they are watching football or relaxing with friends. Very few would dedicate themselves for more than two decades to the world's longest pub crawl. But six friends from the West Midlands have embarked on a special quest to drink in a different pub every Saturday night - and have done so for 28 years. Peter Hill, 56, his father John Hill, 81, John Drew, 48, Kelvin Price, 34, Richard Hill, 30, and Gary Mountain, 54, make up the Black Country Ale Tairsters (BATs) and have been to a total of 16,337 pubs, hotels and wine bars. Cheers! Peter Hill (left), John Drew (middle) and Kelvin Price (right) from the Black Country Ale Tairsters  enjoying a pint at their local . Living the dream: BATs have visited a new pub every Saturday night for 28 years, they've been to more than 16,000 . Originally a team of 20 when it started in 1984, the BATs have now slimmed down to half a dozen men on a serious mission. They have vowed to visit very single 

In [18]:
cnn_train["highlights"][282195]

'The Black Country Ale Tairsters have been to 16,337 pubs, hotels and wine bars .\nThey fill in an elaborate score card and leave a personalised beer mat in every bar .\nWomen are allowed to the pub but not on boozy weekends away .'

In [19]:
cnn_train.tail()

,article,highlights
282192,"The nine-year-old daughter of a black, unarmed...","Rumain Brisbon, 34, was killed after Phoenix p..."
282193,Legalising assisted suicide is a slippery slop...,"Theo Boer, a European assisted suicide watchdo..."
282194,A group calling itself 'The Women of the 99 Pe...,Ohio congressman criticised for 'condoning the...
282195,Most men enjoy a good pint of lager or real al...,The Black Country Ale Tairsters have been to 1...
282196,A Facebook page seeking to preserve the 'Black...,Facebook page supporting tradition gains one m...


In [20]:
cnn_test.describe()

,article,highlights
count,11490,11490
unique,11488,11449
top,"(CNN)Tornadoes, fierce winds and severe thunde...",Investigators have released a handful of photo...
freq,2,3


In [21]:
cnn_test.head()

,article,highlights
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b..."
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...


In [22]:
cnn_test.drop_duplicates(subset=["highlights"], inplace=True)
cnn_test.reset_index(drop=True, inplace=True)

In [23]:
cnn_test.describe()

,article,highlights
count,11449,11449
unique,11449,11449
top,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...
freq,1,1


In [24]:
cnn_test["article"][11448]

"Angus Hawley's brother has spoken of his shock after his brother, the ex-husband of Antonia Kidman, died of a suspected heart attack, age 46, in New York on Saturday. Speaking to Daily Mail Australia on Monday, David Hawley said: 'It's a real shock, he was one of the fittest men I've ever met – he's swimming everyday.' Responding to a question about whether Angus had a history of heart problems, David answered: 'No, no, not that we know of', adding: 'He's so fit, I don't understand.' Scroll down for video . 'He didn't have heart problems' Angus Hawley's brother reveals shock after ex-husband of Antonia Kidman dies from a suspected heart attack in New York after 'returning from a swim'. Angus and Antonia pictured together in 2005 at the Chuan Spa opening in the Langham Hotel . Mr Hawley, who was in New York attending a business conference at the time, collapsed after returning from a swim. 'He did go for a big swim in the morning, he trains very hard,' David said of his brother, who he

In [25]:
cnn_test["highlights"][11448]

"Angus Hawley's brother said his late sibling 'didn't have heart problems'\nHe is reported to have had a suspected heart attack in New York .\nAngus was a father of four children - Lucia, Hamish, James and Sybella .\nHe had all four with Nicole Kidman's sister Antonia before their 2007 split .\nBoth 44-year-old Antonia and Angus, 46, remarried following their divorce .\nAngus' death comes seven months after Dr. Antony Kidman's death .\nNicole and Antonia's father also died of a heart attack in Singapore ."

In [26]:
cnn_val.describe()

,article,highlights
count,13368,13368
unique,13368,13300
top,"(CNN)Share, and your gift will be multiplied. ...",This page includes the show Transcript .\nUse ...
freq,1,16


In [27]:
cnn_val.drop_duplicates(subset=["highlights"], inplace=True)
cnn_val.reset_index(drop=True, inplace=True)

In [28]:
cnn_val.describe()

,article,highlights
count,13300,13300
unique,13300,13300
top,"(CNN)Share, and your gift will be multiplied. ...",Zully Broussard decided to give a kidney to a ...
freq,1,1


In [29]:
cnn_val["article"][13299]

"A teacher and wrestling coach has been charged after allegedly having sex with a high school wrestler. Megan Blair Baker, 25, has been accused of sexual contact with a 17-year-old Sherwood Cass High School student amid reports that she had sex with him on a school bus in January. The teacher, from Urich, Missouri who is believed to be married, was indicted by grand jury on March 20. She was freed on $7,500 bond, KCTV5\xa0reported. Scroll down for video . Teacher and wrestling coach Megan Blair Baker, 25, has been accused of sexual contact with a 17-year-old Sherwood Cass High School student . According to members of the school community who spoke to the local TV station, the teenage boy 'had been bragging' about what happened with the teacher. The boy's father reported the incident to police on February 2. School superintendent Tim Gallagher told Daily Mail Online that Mrs Baker was placed on paid administrative leave when the allegations were first reported. She could face up to four

In [30]:
cnn_val["highlights"][13299]

'Megan Blair Baker, 25, has been accused of sexual contact with a 17-year-old Sherwood Cass High School student .\nShe reportedly had sex with the teenager on a school bus in January .\nMrs Baker was suspended from the school on paid leave and is out on $7,500 bond .'

In [31]:
from datasets import Dataset, DatasetDict

# Convert pandas DataFrames back to Hugging Face datasets
train_dataset = Dataset.from_pandas(cnn_train)
test_dataset = Dataset.from_pandas(cnn_test)
val_dataset = Dataset.from_pandas(cnn_val)


In [32]:
# Create a new DatasetDict
cnn_datasets = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': val_dataset
})

In [33]:
cnn_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 282197
    })
    test: Dataset({
        features: ['article', 'highlights'],
        num_rows: 11449
    })
    validation: Dataset({
        features: ['article', 'highlights'],
        num_rows: 13300
    })
})

In [34]:
cnn_dataset = cnn_datasets.filter(lambda x: len(x["article"].split()) > 100)

Filter:   0%|          | 0/282197 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11449 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13300 [00:00<?, ? examples/s]

In [35]:
cnn_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 281822
    })
    test: Dataset({
        features: ['article', 'highlights'],
        num_rows: 11433
    })
    validation: Dataset({
        features: ['article', 'highlights'],
        num_rows: 13288
    })
})

In [36]:
train_dataset = cnn_dataset["train"]
test_dataset = cnn_dataset["test"]
val_dataset = cnn_dataset["validation"]

## **NOTE:-** Well its always better to test model on smaller dataset

In [37]:
from sklearn.model_selection import train_test_split

data_size = 2000

train_small = train_dataset.select(range(data_size))
test_small = test_dataset.select(range(data_size))
val_small = val_dataset.select(range(data_size))

In [38]:
train_small

Dataset({
    features: ['article', 'highlights'],
    num_rows: 2000
})

In [39]:
cnn_small = DatasetDict({
    'train': train_small,
    'test': test_small,
    'validation': val_small
})


## 3. Preprocessing the data

In [40]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [41]:
inputs = tokenizer("what's going on are you mad?")
inputs

{'input_ids': [0, 12196, 18, 164, 15, 32, 47, 7758, 116, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [42]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['<s>', 'what', "'s", 'Ġgoing', 'Ġon', 'Ġare', 'Ġyou', 'Ġmad', '?', '</s>']

In [43]:
max_input_length = 512
max_target_length = 60

def preprocess_function(examples):
  model_inputs = tokenizer(
      examples["article"],
      max_length=max_input_length,
      truncation=True,)
  labels = tokenizer(
      examples["highlights"],
      max_length=max_target_length,
      truncation=True,)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [44]:
tokenized_datasets_small = cnn_small.map(preprocess_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## 4. Metrics

In [45]:
!pip install rouge_score

In [46]:
!pip install evaluate

In [47]:
import evaluate

rouge_score = evaluate.load("rouge")

In [48]:
!pip install nltk

In [49]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [50]:
from nltk.tokenize import sent_tokenize


def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


print(three_sentence_summary(cnn_dataset["train"][1]["article"]))

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."


In [51]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["article"]]
    return metric.compute(predictions=summaries, references=dataset["highlights"])

In [52]:
import pandas as pd

score = evaluate_baseline(cnn_small["validation"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = {rn: round(score[rn] * 100, 2) for rn in rouge_names}
rouge_dict

{'rouge1': 31.91, 'rouge2': 12.97, 'rougeL': 20.95, 'rougeLsum': 28.85}

## 6. Fine tuning with the Trainer API

In [53]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [2]:
!pip install accelerate -U

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [63]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 1
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets_small["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir="Sumx-test-model",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [71]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    return {k: round(v, 4) for k, v in result.items()}

In [72]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [73]:
# This code didn't worke here cause it was already used once and hence we already have tokenized_datasets and hence it didn't work now

tokenized_datasets_small = tokenized_datasets_small.remove_columns(
    cnn_small["train"].column_names
)

ValueError: Column name ['article', 'highlights'] not in the dataset. Current columns in the dataset: ['input_ids', 'attention_mask', 'labels']

In [74]:
features = [tokenized_datasets_small["train"][i] for i in range(2)]
data_collator(features)

{'input_ids': tensor([[    0,   574,  4524,  ...,    14,    37,     2],
        [    0, 31611,    18,  ...,   169,   480,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[    0, 29345, 10997,   999,  3028,  7312, 20152,  1516,   984,   844,
           448, 13016,    25,    37,  4072,   504,   302,   479, 50118, 22138,
          2701,   161,    37,    34,   117,   708,     7,   856,  3961,  1334,
            39,  1055,   409,   479, 50118, 28243, 20152,    18,  1107,    31,
            78,   292, 10997,  3541,    33,    57,   547,    11,  2416,  1391,
           479,     2,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [    0,   448,  1342,  2368,  4812,  8039,    11,  2561,    32, 15740,
            15,     5,    22,  1990, 35095,  1929,   113, 50118, 40145,  5031,
          1063,  1594,   397,   161,   144,    32,    89,    25,    10,   898,
             9,    22, 40623,   868, 14383, 17130,   113, 

In [75]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets_small["train"],
    eval_dataset=tokenized_datasets_small["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [76]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.147600,1.852769,0.374600,0.159200,0.257800,0.347600


TrainOutput(global_step=250, training_loss=1.1475657958984375, metrics={'train_runtime': 1595.7828, 'train_samples_per_second': 1.253, 'train_steps_per_second': 0.157, 'total_flos': 2167104602112000.0, 'train_loss': 1.1475657958984375, 'epoch': 1.0})

In [77]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

events.out.tfevents.1715766662.41de54ffeca7.3209.0:   0%|          | 0.00/5.80k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

events.out.tfevents.1715768541.41de54ffeca7.3209.2:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

events.out.tfevents.1715766707.41de54ffeca7.3209.1:   0%|          | 0.00/11.7k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Priyanshuchaudhary2425/Sumx-test-model/commit/cb139829db7e1ca59c7532b58b1dacb7c4be84df', commit_message='Training complete', commit_description='', oid='cb139829db7e1ca59c7532b58b1dacb7c4be84df', pr_url=None, pr_revision=None, pr_num=None)